In [1]:
from openai import OpenAI
import openai
import os
import ast
import re
import json
import markdown
from bs4 import BeautifulSoup
import pickle

import requests
from xml.etree import ElementTree
import ast

from collections import defaultdict
from difflib import SequenceMatcher
import numpy as np
from scipy.cluster.hierarchy import fcluster
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from linkml_runtime.loaders.yaml_loader import YAMLLoader
from linkml_runtime.utils.schemaview import SchemaView

In [3]:
openai.api_key = ""

In [4]:
# Initialize the OpenAI client
client = OpenAI(api_key=openai.api_key)#, base_url="https://api.cborg.lbl.gov")

In [5]:
# Define the function to query the OpenAI API
def query_openai(model, role, prompt):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": role, "content": prompt}
            ]
        )
        return response#.choices[0].message['content'] if response.choices else None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
#metadata-elements-comparison.txt


In [7]:
schema_path = "../src/data_sheets_schema/schema/data_sheets_schema.yaml"
# Load schema
#yaml_loader = YAMLLoader()
schema_view = SchemaView(schema_path)

print(f"Schema name: {schema_view.schema.name}")
classes = schema_view.all_classes()
print(f"Classes in the schema: {list(classes.keys())}")

INFO:root:Importing standards_organization_schema as standards_organization_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing standards_schema as standards_schema from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=../src/data_sheets_schema/schema
INFO:root:Importing linkml:types as /Users/marcin/Documents/VIMSS/ontology/bridge2ai/data-sheets-schema/venv/lib/python3.10/site-packages/linkml_runtime/linkml_model/model/schema/types from source ../src/data_sheets_schema/schema/data_sheets_schema.yaml; base_dir=None


Schema name: data-sheets-schema
Classes in the schema: ['NamedThing', 'AnatomicalEntity', 'Organization', 'OrganizationContainer', 'Information', 'FormatDialect', 'Person', 'DatasetProperty', 'DatasetCollection', 'Dataset', 'DataSubset', 'Software', 'Purpose', 'Task', 'AddressingGap', 'Creator', 'FundingMechanism', 'Grantor', 'Grant', 'Instance', 'SamplingStrategy', 'MissingInfo', 'Relationships', 'Splits', 'DataAnomaly', 'ExternalResource', 'Confidentiality', 'ContentWarning', 'Subpopulation', 'Deidentification', 'SensitiveElement', 'InstanceAcquisition', 'CollectionMechanism', 'DataCollector', 'CollectionTimeframe', 'EthicalReview', 'DirectCollection', 'CollectionNotification', 'CollectionConsent', 'ConsentRevocation', 'DataProtectionImpact', 'PreprocessingStrategy', 'CleaningStrategy', 'LabelingStrategy', 'RawData', 'ExistingUse', 'UseRepository', 'OtherTask', 'FutureUseImpact', 'DiscouragedUse', 'ThirdPartySharing', 'DistributionFormat', 'DistributionDate', 'LicenseAndUseTerms', 'I

In [8]:
schema_view

SchemaView(schema=SchemaDefinition({
  'name': 'data-sheets-schema',
  'description': 'A LinkML schema for Datasheets for Datasets.',
  'title': 'data-sheets-schema',
  'see_also': ['https://bridge2ai.github.io/data-sheets-schema'],
  'id': 'https://w3id.org/bridge2ai/data-sheets-schema',
  'imports': ['linkml:types', 'standards_schema', 'standards_organization_schema'],
  'license': 'MIT',
  'prefixes': {'biolink': Prefix({'prefix_prefix': 'biolink', 'prefix_reference': 'https://w3id.org/biolink/vocab/'}),
    'csvw': Prefix({'prefix_prefix': 'csvw', 'prefix_reference': 'http://www.w3.org/ns/csvw#'}),
    'data_sheets_schema': Prefix({
      'prefix_prefix': 'data_sheets_schema',
      'prefix_reference': 'https://w3id.org/bridge2ai/data-sheets-schema/'
    }),
    'datasets': Prefix({'prefix_prefix': 'datasets', 'prefix_reference': 'https://w3id.org/linkml/report'}),
    'dcat': Prefix({'prefix_prefix': 'dcat', 'prefix_reference': 'http://www.w3.org/ns/dcat#'}),
    'example': Prefix

In [9]:
# Get all classes
classes = schema_view.all_classes()
for class_name, class_def in classes.items():
    print(f"Class: {class_name}, Definition: {class_def}")

# Get all slots
slots = schema_view.all_slots()
for slot_name, slot_def in slots.items():
    print(f"Slot: {slot_name}, Definition: {slot_def}")

# Get all enums
enums = schema_view.all_enums()
for enum_name, enum_def in enums.items():
    print(f"Enum: {enum_name}, Definition: {enum_def}")


Class: NamedThing, Definition: ClassDefinition({
  'name': 'NamedThing',
  'description': 'A generic grouping for any identifiable entity',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'slots': ['id', 'category', 'name', 'description', 'subclass_of', 'related_to',
    'contributor_name', 'contributor_github_name', 'contributor_orcid',
    'contribution_date'],
  'class_uri': 'schema:Thing'
})
Class: AnatomicalEntity, Definition: ClassDefinition({
  'name': 'AnatomicalEntity',
  'id_prefixes': ['uberon'],
  'description': 'A subcellular location, cell type or gross anatomical part',
  'from_schema': 'https://w3id.org/bridge2ai/standards-schema',
  'exact_mappings': ['uberon:0001062', 'wikidata:Q4936952'],
  'narrow_mappings': ['ncit:C12219'],
  'is_a': 'NamedThing'
})
Class: Organization, Definition: ClassDefinition({
  'name': 'Organization',
  'description': ('Represents a group or organization related to or responsible for one or more '
     'Bridge2AI standards.

In [10]:
class_details = {}

for class_name, class_def in schema_view.all_classes().items():
    description = class_def.description if class_def.description else 'No description available'
    subsets = class_def.in_subset if class_def.in_subset else []
    attributes = {}
    for attr_name, attr_def in class_def.attributes.items():
        attributes[attr_name] = attr_def.description if attr_def.description else 'No description available'
    
    class_details[class_name] = {
        'subset': subsets,
        'attributes': attributes,
        'description': description
    }

# Print the resulting dictionary for verification
for class_name, details in class_details.items():
    print(f"Class: {class_name}")
    print(f"  Subsets: {details['subset']}")
    print(f"  Description: {details['description']}")
    print(f"  Attributes:")
    for attr_name, attr_desc in details['attributes'].items():
        print(f"    - {attr_name}: {attr_desc}")
    print()

Class: NamedThing
  Subsets: []
  Description: A generic grouping for any identifiable entity
  Attributes:

Class: AnatomicalEntity
  Subsets: []
  Description: A subcellular location, cell type or gross anatomical part
  Attributes:

Class: Organization
  Subsets: []
  Description: Represents a group or organization related to or responsible for one or more Bridge2AI standards.
  Attributes:

Class: OrganizationContainer
  Subsets: []
  Description: A container for Organizations.
  Attributes:

Class: Information
  Subsets: []
  Description: Grouping for datasets and data files
  Attributes:

Class: FormatDialect
  Subsets: []
  Description: Additional format information for a file
  Attributes:
    - comment_prefix: No description available
    - delimiter: No description available
    - double_quote: No description available
    - header: No description available
    - quote_char: No description available

Class: Person
  Subsets: []
  Description: An individual human being.
  Attr

In [11]:
# Prepare the class details as a formatted string for the prompt
class_details_str = "\n".join(
    f"Class name: {class_name}\n"
    f"  Subsets: {', '.join(details['subset'])}\n"
    f"  Description: {details['description']}\n"
    f"  Attributes:\n" +
    "".join(f"    {attr_name}: {attr_desc}\n" for attr_name, attr_desc in details['attributes'].items())
    for class_name, details in class_details.items()
)


In [22]:
import pandas as pd

# Try reading the file with 'latin-1' encoding
try:
    datacards_structure = pd.read_csv("./metadata-elements__datacards_breakdown.txt", sep="\t", header=0, index_col=0, encoding='latin-1')
    print(datacards_structure.shape)
except UnicodeDecodeError:
    print("Failed to read file with 'latin-1' encoding.")
    # If the above fails, try reading the file with 'windows-1252' encoding
    try:
        datacards_structure = pd.read_csv("./metadata-elements__datacards_breakdown.txt", sep="\t", header=0, index_col=0, encoding='windows-1252')
        print(datacards_structure.shape)
    except UnicodeDecodeError:
        print("Failed to read file with 'windows-1252' encoding.")


datacards_structure.reset_index(inplace=True) 
datacards_structure

(298, 6)


,Count,Reference code,Question or field,Metadata reference code,Metadata description,Is controlled value,Metadata code
0,1.0,Summary.1,Dataset Name (Acronym),Summary.1,Name of the dataset,False,NaN
1,2.0,Summary.2,Write a short summary describing your dataset ...,Summary.2,Short summary describing the dataset (limit 20...,False,NaN
2,3.0,Summary.3,Provide a link to the dataset:\nDATASET LINK\n...,Summary.3,Link to the dataset,False,NaN
3,4.0,Summary.4,Select one role per Data Card Author:\n\n(Usag...,Summary.4.1,Name of the authors of the data card,False,NaN
4,NaN,NaN,NaN,Summary.4.2,Affiliation of the authors of the data card,False,NaN
...,...,...,...,...,...,...,...
293,171.0,Known_applications_and_benchmarks.3,Provide a description of the evaluation proces...,Known_applications_and_benchmarks.3,NaN,NaN,Provide a description of the evaluation proces...
294,172.0,Known_applications_and_benchmarks.4,Duplicate this row as necessary for each model...,Known_applications_and_benchmarks.4,NaN,NaN,Duplicate this row as necessary for each model...
295,173.0,Known_applications_and_benchmarks.5,Provide a description of the expected performa...,Known_applications_and_benchmarks.5,NaN,NaN,Provide a description of the expected performa...
296,174.0,Terms_of_art.1.Concepts_and_definitions_refere...,Use this space to include the expansions and d...,Terms_of_art.1.Concepts_and_definitions_refere...,NaN,NaN,Use this space to include the expansions and d...


In [29]:
llm_prompt_template = """
You are a biomedical semantic curator and engineer working on comparing, aligning, and merging two complex biomedical schemas for health data.

You are given a Data Cards of class details from another schema. 
Your tasks are as follows:

1. **Identify Relevant Class Attributes**: Based on the other schema consisting of Class, Subsets, Description, and Attributes, find the most relevant Class Attribute for this Metadata Code.
   - If multiple class attributes are found to match, list all. This may involve different Classes and multiple Attributes from the same Class.
   - If no reasonable match is found, report "No match" in the 'Closeness' field and leave the others empty.
3. **Describe Match Type**: Use the Closeness field to describe the match type: "exact", "close", or "partial".

Make sure that the entire schema is considered and that all exact and close matches are reported. Double check to make sure this is the case.

Format the output as in the example below, ensuring that matches to different Class Attributes for this metadata reference code will have separate output sections in this output format.
Do not add any other text or formatting to the output and make sure to adhere to this output format including naming, structuring, and ordering of fields.

Example exact matches (hypothetical matches that may not actually be in this data):
**Data Cards entry:** "dataset_description"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "description"
**Data Cards entry:** "is_static_or_dynamic"\n  - **Closeness:** "exact"\n  - **Class:** "Dataset"\n  - **Attribute:** "updates"

Example close matches:
**Data Cards entry:** "dataset_description"\n  - **Closeness:** "close"\n  - **Class:** "Purpose"\n  - **Attribute:** "response"
**Data Cards entry:** "has_audit"\n  - **Closeness:** "close"\n  - **Class:** "Dataset"\n  - **Attribute:** "ethical_reviews"'

Example partial matches:
**Data Cards entry:** "has_audit"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "data_protection_impacts"
**Data Cards entry:** "who_audit"\n  - **Closeness:** "partial"\n  - **Class:** "ExistingUse"\n  - **Attribute:** "description"
**Data Cards entry:** "who_audit"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "data_collectors"

Example no matches entry:
**Data Cards entry:** "destruction_date"\n  - **Closeness:** "No match"\n  - **Class:** ""\n  - **Attribute:** ""


Output format with variable to complete by LLM:

**Data Cards entry:** "<metadata reference code>"
  - **Closeness:** "<closeness1>"
  - **Class:** "<class_name1>"
  - **Attribute:** "<attribute name1>"
**Data Cards entry:** "<metadata reference code>"
  - **Closeness:** "<closeness2>"
  - **Class:** "<class_name1>"
  - **Attribute:** "<attribute name2>"
**Data Cards entry:** "<metadata reference code>"
  - **Closeness:** "<closeness3>"
  - **Class:** "<class_name2>"
  - **Attribute:** "<attribute name3>"
---

Here is the Data Cards data row for the specific metadata code and its other associated fields for context:

- Metadata reference Code: {reference_code}
- Question: {question}
- Metadata Description: {metadata_description}


Here are the full class details from the other schema:

{class_details}

"""

In [30]:
datacards_structure.index

RangeIndex(start=0, stop=298, step=1)

In [31]:
llm_results = []

# Function to strip HTML/Markdown tags
def strip_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Loop over Data Cards structure entries, fill out the prompt template, and submit to the LLM
for index, row in datacards_structure.iterrows():
    reference_code = row['Metadata reference code']
    question = row['Question or field']
    metadata_description = row['Metadata description']

    if pd.isna(reference_code) or pd.isna(question):
        # Ensure index is an integer
        index = int(index)
        # Search for the first earlier row with non-NaN values for Reference Code and Question
        for prev_index in range(index - 1, -1, -1):
            prev_row = datacards_structure.iloc[prev_index]
            if pd.notna(prev_row['Metadata reference code']) and pd.notna(prev_row['Question or field']):
                reference_code = prev_row['Metadata reference code']
                question = prev_row['Question or field']
                break
    
    prompt = llm_prompt_template.format(
        reference_code=reference_code,
        question=question,
        metadata_description=metadata_description,
        class_details=class_details_str
    )
    
    # Call to LLM (Replace with actual LLM call)
    result = query_openai("gpt-4", "user", prompt)  # Uncomment this for actual LLM call
    # result = f"Simulated result for prompt: {prompt}"  # Simulated result for demonstration

    # print(prompt)  # For debugging
    print(result)
    llm_results.append(result)
    
# Output the results for verification
for result in llm_results:
    print(result)
    print("------")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A95GKh2j5wqaKKv0rFV5a6cVwAtLn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Data Cards entry:** "Summary.1"\n  - **Closeness:** "No match"\n  - **Class:** ""\n  - **Attribute:** ""', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726728544, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=33, prompt_tokens=5447, total_tokens=5480, completion_tokens_details={'reasoning_tokens': 0}))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A95GNANU2l3Zu1JosTZCstoCSxXbc', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Data Cards entry:** "Summary.2"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "purposes"\n**Data Cards entry:** "Summary.2"\n  - **Closeness:** "partial"\n  - **Class:** "Task"\n  - **Attribute:** "response"\n**Data Cards entry:** "Summary.2"\n  - **Closeness:** "partial"\n  - **Class:** "Purpose"\n  - **Attribute:** "response"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726728547, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=109, prompt_tokens=5521, total_tokens=5630, completion_tokens_details={'reasoning_tokens': 0}))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

ChatCompletion(id='chatcmpl-A963zxu7QFZ0ACB1R198aailBZ5zG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Data Cards entry:** "Known_applications_and_benchmarks.5"\n  - **Closeness:** "partial"\n  - **Class:** "ExistingUse"\n  - **Attribute:** "description"\n**Data Cards entry:** "Known_applications_and_benchmarks.5"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "used_software"\n**Data Cards entry:** "Known_applications_and_benchmarks.5"\n  - **Closeness:** "partial"\n  - **Class:** "DatasetProperty"\n  - **Attribute:** "used_software"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726731623, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=132, prompt_tokens=5544, total_tokens=5676, completion_tokens_details={'reasoning_tokens': 0}))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A9647nWRnnU6q1t0aknv1NmyUl23d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Data Cards entry:** "Terms_of_art.1.Concepts_and_definitions_referenced_in_this_data_card.1"\n  - **Closeness:** "No match"\n  - **Class:** ""\n  - **Attribute:** ""', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726731631, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=47, prompt_tokens=5645, total_tokens=5692, completion_tokens_details={'reasoning_tokens': 0}))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-A964B19P5RbBgVwTNZVz1dW8cQO0F', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Data Cards entry:** "Reflections_on_data.1"\n  - **Closeness:** "close"\n  - **Class:** "Dataset"\n  - **Attribute:** "content_warnings"\n**Data Cards entry:** "Reflections_on_data.1"\n  - **Closeness:** "close"\n  - **Class:** "ContentWarning"\n  - **Attribute:** "warnings"\n**Data Cards entry:** "Reflections_on_data.1"\n  - **Closeness:** "partial"\n  - **Class:** "Dataset"\n  - **Attribute:** "confidential_elements"\n**Data Cards entry:** "Reflections_on_data.1"\n  - **Closeness:** "partial"\n  - **Class:** "Confidentiality"\n  - **Attribute:** "description"', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1726731635, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=162, prompt_tokens=5516, total_tokens=5678, c

In [32]:
import pandas as pd

# Function to safely extract data between two markers
def extract_between(content, start=None, end=None):
    try:
        if start and end:
            return content.split(start, 1)[1].split(end, 1)[0].strip()
        elif start:
            return content.split(start, 1)[1].strip()
        elif end:
            return content.split(end, 1)[0].strip()
    except IndexError:
        return "Not Found"

# Parse the LLM responses and extract the required data
parsed_data = []

for response in llm_results:
    try:
        response_content = response.choices[0].message.content
        # Split the response content into individual Data Cards entries
        entries = response_content.split('**Data Cards entry:**')[1:]  # Split and remove the first empty split section
        
        for entry in entries:
            # Process individual Data Cards entries
            entry_text = extract_between(entry, '"', '"')
            closeness = extract_between(entry, '- **Closeness:** "', '"')
            class_name = extract_between(entry, '- **Class:** "', '"')
            attribute = extract_between(entry, '- **Attribute:** "', '"')
            
            parsed_data.append({
                "Data Cards entry": entry_text,
                "Closeness": closeness,
                "Class": class_name,
                "Attribute": attribute
            })

    except Exception as e:
        print(f"An error occurred while parsing response: {e}")

# Create a DataFrame with the extracted data
df = pd.DataFrame(parsed_data, columns=["Data Cards entry", "Closeness", "Class", "Attribute"])

# Determine if each Data Cards entry has an exact match or only partial matches
grouped = df.groupby('Data Cards entry')['Closeness'].apply(list).reset_index()

# Create HasExact and OnlyPartial columns
grouped['HasExact'] = grouped['Closeness'].apply(lambda x: 'yes' if 'exact' in x else 'no')
grouped['OnlyPartial'] = grouped['Closeness'].apply(lambda x: 'yes' if all(c in ['partial', 'No match'] for c in x) else 'no')

# Merge the new columns back into the original DataFrame
df = df.merge(grouped[['Data Cards entry', 'HasExact', 'OnlyPartial']], on='Data Cards entry', how='left')

# Save the DataFrame to a TSV file
df.to_csv('datacards_into_datasheet_codes_llm_results_v1.tsv', sep='\t', index=False)

df

,Data Cards entry,Closeness,Class,Attribute,HasExact,OnlyPartial
0,Summary.1,No match,,,no,yes
1,Summary.2,partial,Dataset,purposes,no,yes
2,Summary.2,partial,Task,response,no,yes
3,Summary.2,partial,Purpose,response,no,yes
4,Summary.3,partial,Dataset,external_resources,no,yes
...,...,...,...,...,...,...
1052,Terms_of_art.1.Concepts_and_definitions_refere...,No match,,,no,yes
1053,Reflections_on_data.1,close,Dataset,content_warnings,no,no
1054,Reflections_on_data.1,close,ContentWarning,warnings,no,no
1055,Reflections_on_data.1,partial,Dataset,confidential_elements,no,no


In [33]:
# Save the object to a file
with open('data_vs_data_card_codes_alignment_raw_v1.pkl', 'wb') as file:
    pickle.dump(llm_results, file)

In [34]:
#with open('data_vs_health_sheet_code_alignment_raw_v4.pkl', 'rb') as file:
#    llm_results = pickle.load(file)

In [35]:
df

,Data Cards entry,Closeness,Class,Attribute,HasExact,OnlyPartial
0,Summary.1,No match,,,no,yes
1,Summary.2,partial,Dataset,purposes,no,yes
2,Summary.2,partial,Task,response,no,yes
3,Summary.2,partial,Purpose,response,no,yes
4,Summary.3,partial,Dataset,external_resources,no,yes
...,...,...,...,...,...,...
1052,Terms_of_art.1.Concepts_and_definitions_refere...,No match,,,no,yes
1053,Reflections_on_data.1,close,Dataset,content_warnings,no,no
1054,Reflections_on_data.1,close,ContentWarning,warnings,no,no
1055,Reflections_on_data.1,partial,Dataset,confidential_elements,no,no


In [36]:
df[df['Closeness'] == 'exact']

,Data Cards entry,Closeness,Class,Attribute,HasExact,OnlyPartial
6,Summary.4.1,exact,Person,affiliation,yes,no
7,Summary.4.1,exact,Person,affiliation,yes,no
8,Summary.4.1,exact,Creator,affiliation,yes,no
12,Authorship.1.Publishers.1,exact,Organization,-,yes,no
39,Authorship.1.Publishers.3.1,exact,Person,affiliation,yes,no
...,...,...,...,...,...,...
980,Annotations_and_labeling.2,exact,LabelingStrategy,description,yes,no
987,Annotations_and_labeling.5,exact,Task,response,yes,no
1035,Sampling_methods.1,exact,SamplingStrategy,strategies,yes,no
1036,Sampling_methods.2,exact,SamplingStrategy,strategies,yes,no


In [37]:
# Group by 'Data Cards entry' and check for the presence of 'exact' closeness
unique_entries_with_exact = df.groupby('Data Cards entry').apply(lambda x: 'exact' in x['Closeness'].values)
exact_entries = unique_entries_with_exact[unique_entries_with_exact].index

# Count the number of unique Data Cards entries with an exact match
count_exact_entries = len(exact_entries)
count_exact_entries

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_12003/3818477385.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_entries_with_exact = df.groupby('Data Cards entry').apply(lambda x: 'exact' in x['Closeness'].values)


51

In [38]:
# Group by 'Data Cards entry' and check for the presence of 'exact' closeness
unique_entries = df.groupby('Data Cards entry').apply(lambda x: 'exact' not in x['Closeness'].values)
non_exact_entries = unique_entries[unique_entries].index

# Count the number of unique Data Cards entries without an exact match
count_non_exact_entries = len(non_exact_entries)

count_non_exact_entries

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_12003/313070652.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_entries = df.groupby('Data Cards entry').apply(lambda x: 'exact' not in x['Closeness'].values)


124

In [42]:
df[df['Class'] == '']

,Data Cards entry,Closeness,Class,Attribute,HasExact,OnlyPartial
0,Summary.1,No match,,,no,yes
303,Example_of _data_points.1,No match,,,no,yes
307,Example_of _data_points.2.1,No match,,,no,no
308,Example_of _data_points.2.1,No match,,,no,no
322,Example_of _data_points.3.1,No match,,,no,no
437,Access_rentention_and_wipeout.3.Wipeout_and_de...,No match,,,no,yes
442,Access_rentention_and_wipeout.3.Wipeout_and_de...,No match,,,no,yes
539,Provenance.1.Collection.5.1,No match,,,yes,no
778,Extended_use.1.Use_with_other_data.1,No match,,,no,yes
798,Extended_use.2.Forking_and_sampling.1,No match,,,no,yes
